In [1]:
import matplotlib.pyplot as plt
import numpy as np
from commpy.modulation import QAMModem

from scripts.dsp import pulseShape, firFilter, decimate, symbolSync, resample
from scripts.models import phaseNoise, KramersKronigRx, linFiberCh
from scripts.tx import simpleWDMTx
from scripts.core import parameters
from scripts.equalization import edc, mimoAdaptEqualizer
from scripts.carrierRecovery import cpr
from scripts.metrics import fastBERcalc, monteCarloGMI, monteCarloMI, signal_power
from scripts.plot import pconst

import scipy.constants as const

In [2]:
plt.rcParams['font.size'] = 14
plt.rcParams['lines.linewidth'] = 2

# Simulation of coherent transmission

## **Transmitter**
**Simulation of a single polarization optical signal transmission**

In [3]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 400000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
paramTx.alphaRRC = 0.01        # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 0       # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
FO = paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12        # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 2

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate

In [4]:
# generate optical signal signal
sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

# simulate linear signal propagation
sigCh_0 = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)
# addition of multiple dispersions
sigCh_1 = linFiberCh(sigCh_0, 25, alpha, D, Fc, Fs)
sigCh_2 = linFiberCh(sigCh_0, 35, alpha, D, Fc, Fs)
sigCh_3 = linFiberCh(sigCh_0, 55, alpha, D, Fc, Fs)

# receiver detection and demodulation
Fc = paramTx.Fc
Ts = 1 / Fs
mod = QAMModem(m=paramTx.M)

freqGrid = paramTx.freqGrid
print(
    "Demodulating channel #%d , fc: %.4f THz, λ: %.4f nm\n"
    % (
        chIndex,
        (Fc + freqGrid[chIndex]) / 1e12,
        const.c / (Fc + freqGrid[chIndex]) / 1e-9,
    )
)

symbTx = symbTx_[:, :, chIndex]
Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

print(
    "Local oscillator P: %.2f dBm, lw: %.2f kHz, FO: %.2f MHz"
    % (Plo_dBm, lw / 1e3, FO / 1e6)
)

channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 16000.00 MHz


# Phase-retrieval stage

**Simulation of an ideal direct sensing optical receiver**

In [5]:
Amp_0 = np.abs(sigCh_0)
Amp_1 = np.abs(sigCh_1)
Amp_2 = np.abs(sigCh_2)
Amp_3 = np.abs(sigCh_3)

# resampling to ADC sampling rate
Amp_0 = resample(Amp_0, paramADC)
Amp_1 = resample(Amp_1, paramADC)
Amp_2 = resample(Amp_2, paramADC)
Amp_3 = resample(Amp_3, paramADC)

newFs = paramADC.SpS_out*paramTx.Rs

In [6]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, MaxPool1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [7]:
# Neural network training

N = 256  # number of input amplitude samples to the NN

sigCh_0 = resample(sigCh_0, paramADC)

Amp_0 = Amp_0/np.sqrt(signal_power(Amp_0))
Amp_1 = Amp_1/np.sqrt(signal_power(Amp_1))
Amp_2 = Amp_2/np.sqrt(signal_power(Amp_2))
Amp_3 = Amp_3/np.sqrt(signal_power(Amp_3))

sigPhase = np.angle(sigCh_0)  # get signal phase samples (labels) (L,)

# get signal amplitude samples (L,)
sigAmp_0 = np.pad(Amp_0, (int(N/2), int(N/2)), 'constant')
sigAmp_1 = np.pad(Amp_1, (int(N/2), int(N/2)), 'constant')
sigAmp_2 = np.pad(Amp_2, (int(N/2), int(N/2)), 'constant')
sigAmp_3 = np.pad(Amp_3, (int(N/2), int(N/2)), 'constant')

# create set of input features
X_train = np.zeros((len(sigPhase), 4*N))  # (L,N)

for indPhase in range(len(sigPhase)):
    X_train[indPhase] = np.concatenate((sigAmp_0[indPhase:N+indPhase], sigAmp_1[indPhase:N+indPhase], sigAmp_2[indPhase:N+indPhase], sigAmp_3[indPhase:N+indPhase]))


# create set of phase labels
y_train = sigPhase.copy()

C:\Users\SILASJ~1\AppData\Local\Temp/ipykernel_16496/1867627630.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[indPhase] = np.concatenate((sigAmp_0[indPhase:N+indPhase], sigAmp_1[indPhase:N+indPhase], sigAmp_2[indPhase:N+indPhase], sigAmp_3[indPhase:N+indPhase]))


In [ ]:
#print(X_train.shape, y_train.shape)

(200000, 1024) (200000,)


In [ ]:
# define neural network model

# MLP tests
stop = EarlyStopping(monitor='val_loss', patience=5)
#rlronp = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

model = Sequential()

# MLP model
# model.add(BatchNormalization())
# # Input Layer
# model.add(Dense(100, input_shape=(4*N,), activation='relu'))
# model.add(Dropout(0.5))
# # Hidden layer
# model.add(BatchNormalization())
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.1))
# # Hidden layer
# model.add(BatchNormalization())
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
# # Output layer
# model.add(Dense(1, activation='linear'))

# RNN hybrid model
# model.add(layers.LSTM(64, return_sequences=True, input_shape=(4*N, 1)))
# # Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(16))
# # Add a Dense layer with 10 units.
# model.add(BatchNormalization())
# model.add(Dense(16, activation='relu'))
# model.add(Dense(1, activation='linear'))

# CNN model
# model.add(layers.Conv1D(32, 2, activation='relu', input_shape=(4*N, 1)))
# model.add(layers.MaxPooling1D((2)))
# model.add(layers.Conv1D(64, (3), activation='relu'))
# model.add(layers.MaxPooling1D((2)))
# model.add(layers.Conv1D(64, (3), activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(16, activation='relu'))
# model.add(Dense(1, activation='linear'))

model.compile(loss='msle', optimizer='adam')
model.fit(X_train, y_train, epochs=100, callbacks=[stop], validation_split=0.3, batch_size=500)

model.summary()

cw1 = model.layers[1].get_weights()
plt.plot(cw1[0])

#model.save('NN_models/testModel_SpS_'+str(paramADC.SpS_out))